# Realtime Voice Translation with Cerebras AI + LiveKit

Experience ultra-fast realtime voice translation powered by Cerebras AI's inference and LiveKit's voice infrastructure. Translate conversations to any language in real-time with sub-second latency.

## Prerequisites

You'll need three API keys:

1. **Cerebras API Key** - Get yours at [https://cloud.cerebras.ai](https://cloud.cerebras.ai)
2. **OpenAI API Key** - For speech-to-text (Whisper) and text-to-speech. Get it at [https://platform.openai.com](https://platform.openai.com)
3. **LiveKit Credentials** - For WebRTC audio streaming

### Getting LiveKit Credentials

Follow the complete setup guide at:
**[https://inference-docs.cerebras.ai/integrations/livekit](https://inference-docs.cerebras.ai/integrations/livekit)**

Quick steps:
1. Sign up at [LiveKit Cloud](https://cloud.livekit.io/)
2. Create a new project
3. Go to **Settings → Keys** and generate an API key pair
4. Copy your:
   - **LIVEKIT_URL** (starts with `wss://`)
   - **LIVEKIT_API_KEY**
   - **LIVEKIT_API_SECRET**

## Running the Translation Agent - Start Jupyter Notebook

Start a new terminal and run the following commands:

```bash
# Install Jupyter
pip3 install jupyter
```

Launch Jupyter with the correct Python kernel:

```bash
# Register the kernel
python3 -m ipykernel install --user --name=translation-agent --display-name="Python 3.12 (translation-agent)"

# Start Jupyter
jupyter notebook translation_agent.ipynb
```

**Important:** When you open the notebook, make sure to select the **"Python 3.12 (translation-agent)"** kernel from the kernel menu (top right).

### Why Jupyter in a Browser?

Running Jupyter in a browser (not VS Code or other IDEs) ensures proper microphone access via the LiveKit widget. The browser's native WebRTC support provides the best experience for realtime voice interaction.

## 1. Install Dependencies

Run this cell to install the necessary packages (uncomment if needed).

In [ ]:
# Uncomment and run if packages are not installed
!pip3 install "livekit-agents[openai,silero]>=1.3.0" python-dotenv -q

## 2. Setup & Credentials

Set your API keys here. You can also use a `.env` file in the same directory.

In [ ]:
import os
from dotenv import load_dotenv

# Load from .env file if present
load_dotenv()

# Or set credentials directly (uncomment and fill in):
# os.environ["CEREBRAS_API_KEY"] = "csk-..."

# os.environ["OPENAI_API_KEY"] = ""  # For STT/TTS
# os.environ["LIVEKIT_URL"] = "wss://your-project.livekit.cloud"
# os.environ["LIVEKIT_API_KEY"] = "API..."
# os.environ["LIVEKIT_API_SECRET"] = "..."

# Verify credentials are set
required = ["OPENAI_API_KEY", "CEREBRAS_API_KEY", "LIVEKIT_URL", "LIVEKIT_API_KEY", "LIVEKIT_API_SECRET"]
missing = [k for k in required if not os.getenv(k)]
if missing:
    print(f"⚠️  Missing environment variables: {missing}")
else:
    print("✅ All credentials loaded")

In [ ]:
import logging
import os
from livekit.agents import (
    AutoSubscribe,
    JobContext,
    WorkerOptions,
    AgentSession,
)
from livekit.agents.voice import Agent as VoiceAgent
from livekit.plugins import openai, silero

# Suppress verbose logging - only show warnings and above
logging.getLogger("livekit").setLevel(logging.WARNING)
logging.getLogger("livekit.agents").setLevel(logging.WARNING)

# Target language for translation
TARGET_LANGUAGE = "Spanish"  # Change this to your desired target language

def get_translation_prompt(target_language: str) -> str:
    """Generate system prompt for translation."""
    return f"""You are a real-time translator. Your task is to translate spoken text to {target_language}.

Rules:
1. Translate the input text accurately to {target_language}
2. Preserve the tone and intent of the original message
3. Keep translations natural and conversational
4. If the input is already in {target_language}, repeat it with minor improvements if needed
5. Do NOT add explanations or commentary - just translate
6. Respond ONLY with the translated text"""


async def entrypoint(ctx: JobContext):
    """LiveKit agent entrypoint."""
    # Connect to the room
    await ctx.connect(auto_subscribe=AutoSubscribe.AUDIO_ONLY)
    
    print(f"🎤 Connected to room: {ctx.room.name}")
    
    # Create Cerebras LLM client (OpenAI-compatible) with proper headers
    cerebras_llm = openai.LLM(
        model="llama-3.1-8b",
        base_url="https://api.cerebras.ai/v1",
        api_key=os.environ["CEREBRAS_API_KEY"],
        extra_headers={"X-Cerebras-3rd-Party-Integration": "realtime-translation"}
    )
    
    # Create voice agent with translation instructions
    agent = VoiceAgent(
        instructions=get_translation_prompt(TARGET_LANGUAGE),
    )
    
    # Create session with STT/LLM/TTS/VAD components
    session = AgentSession(
        vad=silero.VAD.load(),
        stt=openai.STT(model="whisper-1"),
        llm=cerebras_llm,
        tts=openai.TTS(model="tts-1", voice="alloy"),
    )
    
    # Start session with agent and room
    await session.start(agent=agent, room=ctx.room)
    
    # Greet the user
    await session.say(f"Translation agent ready. I will translate to {TARGET_LANGUAGE}. Please speak.")


print(f"✅ Agent defined. Target language: {TARGET_LANGUAGE}")

## 4. Run the Agent

Run this cell to start the agent. It will display an **embedded microphone widget**.

### How it works:
1. An embedded audio widget appears below (microphone access is requested automatically)
2. Click "Allow" when your browser asks for microphone permission
3. Speak into your microphone - the agent will translate and respond
4. To stop: interrupt this cell (Kernel → Interrupt) or restart the kernel

**Note:** This cell blocks until you stop it. The agent runs continuously while this cell executes.

## 5. Change Target Language (Optional)

To translate to a different language, change `TARGET_LANGUAGE` in cell 3 and re-run cells 3 and 4.

Supported languages:
- English
- Spanish (Español)
- German (Deutsch)
- French (Français)
- Italian (Italiano)
- Portuguese (Português)
- Japanese (日本語)
- Chinese (中文)

In [ ]:
from livekit.agents import jupyter

# Run the agent inside the notebook
# This cell will display an embedded microphone widget and keep running while the agent is active
jupyter.run_app(
    WorkerOptions(entrypoint_fnc=entrypoint)
)